# <span style='color:SlateBlue'>Projeto Módulo 2</span>

##  <span style='color:LightSeaGreen'>Sumário</span>
1. [Descrição](#descr)
2. [Preprando ambiente](#env)
3. [Carregando arquivos](#load)
4. [Objeto grafo](#obj)
5. [Criando o grafo](#graph)
6. [Funções](#functions)
    - [Exibir número de seguidores](#followers)
    - [Exibir pessoas que o usuário segue](#following)
    - [Ordernar lista stories](#stories)
    - [Encontrar top k influencers](#topk)
    - [Encontrar o caminho](#path)
7. [Testes](#methods)

# <span style='color:LightSeaGreen'>Descrição</span> <a id='descr'></a>

Para esse projeto nós criaremos uma rede social baseada no Instagram onde teremos um grafo direcionado, já que posso seguir alguém que não me segue. Além disso, teremos conexões que serão melhores amigos e outras que serão conexão comuns. Logo, teremos um grafo direcionado e ponderado.  
O objetivo será criar algumas funções relacionadas ao grafo e a rede social:  
- Exibir número de seguidores
- Exibir quantidades de pessoas que o usuário segue
- Ordenar a lista de Stories, ou seja, melhores amigos primeiro e depois conexões comuns ordenadas por ordem alfabética -> [melhores amigos em ordem alfabetica , amigos em ordem alfabetica]
- Encontrar top k influencers, ou seja, k pessoas que mais tem seguidores da rede
- Encontrar o caminho entre uma pessoa e outra na rede

# <span style='color:LightSeaGreen'>Preprando ambiente</span> <a id='env'></a>

In [14]:
import pandas as pd
import math

# <span style='color:LightSeaGreen'>Carregando arquivos</span> <a id='load'></a>

In [40]:
# show files in data directory
#!dir data
# read csv files in a pandas data frame
connections = pd.read_csv('data/conexoes.csv', header=None)
users = pd.read_csv('data/usuarios.csv', header=None)
# rename data frame titles
connections.columns = ['follower','following','weight']
users.columns = ['name','username']
# merging the two dataframes
connections.insert(0,'name',None)
for index, name in connections.follower.items():
    connections['name'][index] = users[users.username == name]['name'].item()

<ipython-input-40-a3a3a2398954>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connections['name'][index] = users[users.username == name]['name'].item()


# <span style='color:LightSeaGreen'>Objeto grafo</span> <a id='obj'></a>

In [74]:
class Graph():

    def __init__(self) -> None:
        self.network = {}
        self.profiles = {}

    def addNode(self, name, username):
        self.network[username] = {}
        self.profiles[username] = {
            'Name': name,
            'Followers': 0,
            'Following': 0}

    def connectNode(self, origin, destiny, weight=1):
        
        if destiny not in self.network.get(origin):
            # make connection
            self.network[origin][destiny] = weight
            # increase from origin +1 following
            self.profiles[origin]['Following'] += 1
            # increse from destiny + 1 follower
            self.profiles[destiny]['Followers'] += 1
    
    def showFollowers(self, username):
        return f'Seguidores de {self.profiles[username]["Name"]} : {self.profiles[username]["Followers"]}'

    def showFollowing(self, username):
        return f'Pessoas que {self.profiles[username]["Name"]} segue: {self.profiles[username]["Following"]}'
    
    # BFS algorithm to find path
    def showPath(self, origin, destiny):
        queue = [origin]
        visited = []
        predecessor = {origin: None}
        
        while len(queue) > 0:
            currentNode = queue.pop(0)
            visited.append(currentNode)
            for adjacent in self.network[currentNode].keys():

                if adjacent not in queue + visited:
                    predecessor[adjacent] = currentNode
                    queue.append(adjacent)

                if adjacent == destiny:
                    path = [destiny]
                    while currentNode is not None:
                        path.append(currentNode)
                        currentNode = predecessor[currentNode]
                    path.reverse()
                    result = ['->'] * (len(path) * 2 - 1)
                    result[0::2] = path
                    fullPath = ' '.join([str(elem) for elem in result])
                    #originName = users.loc[users['username']==origin].Name.item()
                    #destinyName = users.loc[users['username']==destiny].Name.item()
                    return f'Path from {self.profiles[origin]["Name"]} to {self.profiles[destiny]["Name"]}: {fullPath}'

        return False

    def storiesOrder(self, username):
        firstSort = {key: value for key, value in sorted(self.network[username].items())}
        secondSort = [key for key, value in sorted(firstSort.items(), reverse=True, key=lambda item: item[1])]
        return f'Ordem dos Stories de {self.profiles[username]["Name"]}:\n {secondSort}'
    
    def findInfluencers(self, k):
        # add first node to queue
        queue = [next(iter(self.profiles))]
        # create visited nodes list
        visited = []
        # create influencers dicitonary
        influencers = {}
        # while the list is not empty
        while len(queue) > 0:
            # extract the first element of the queue
            currentNode = queue.pop(0)
            # add this element into the visited list
            visited.append(currentNode)
            currentFollowerNumber = self.profiles[currentNode]['Followers']
            # verify if list is empty
            if not influencers:
                influencers[currentNode] = currentFollowerNumber
            # get the last element from influencer dict
            leastInfluencer = min(influencers.items(), key=lambda item: item[1])[1]
            # verify if list is full and the new element is bigger
            if len(influencers) == k and currentFollowerNumber > leastInfluencer:
                influencers.popitem()
            # add item and sort values
            if len(influencers) < k :
                influencers[currentNode] = currentFollowerNumber
                influencers = {key: value for key, value in sorted(influencers.items(), reverse=True, key=lambda item: item[1])}
            # visit the adjacent elements of this node
            for adjacent in self.network[currentNode].keys():
                # node not in visited list and in queue
                if adjacent not in queue + visited:
                    # add this adjacent node to queue
                    queue.append(adjacent)
        return f'Top {k} Influencers: {influencers}'


# <span style='color:LightSeaGreen'>Criando o grafo</span> <a id='graph'></a>

In [75]:
g = Graph()

# add unique rows by rowname ID
for row in users.iterrows():
    # row[0] -> index;  row[1] -> tuple values
    # row[1][0] -> name;  row[1][1] -> username
    g.addNode(name = row[1][0], username = row[1][1])

# connect the rows
for row in connections.iterrows():
    # row[0] -> index;  row[1] -> tuple values
    # row[1][1] -> follower;  row[1][2] -> following;  row[1][3] -> weight
    g.connectNode(origin=row[1][1], destiny=row[1][2], weight=row[1][3])

helena42 alice43
helena42 gustavo16
helena42 ana_clara30
helena42 mariana5
helena42 caua11
helena42 ana_julia22
helena42 heloisa37
helena42 melissa42
helena42 davi48
helena42 sophia31
helena42 calebe49
helena42 lavinia36
helena42 nicolas4
helena42 rafael38
helena42 matheus6
helena42 pietro33
alice43 felipe49
alice43 benicio10
laura29 clara1
laura29 eloa17
laura29 joao43
laura29 esther32
laura29 isadora45
laura29 heitor5
laura29 matheus6
laura29 enzo23
laura29 vitor38
laura29 enzo_gabriel23
laura29 joaquim33
laura29 catarina14
laura29 alice43
laura29 marina41
laura29 mariana5
laura29 calebe49
laura29 daniel12
laura29 pietro33
laura29 maria_cecilia32
laura29 joao_lucas7
laura29 lavinia36
laura29 joao_miguel1
laura29 elisa20
manuela19 leonardo7
manuela19 miguel1
manuela19 laura29
manuela19 ana_julia22
manuela19 joao_lucas7
manuela19 heitor5
manuela19 caua11
manuela19 samuel45
valentina26 lucas26
valentina26 enzo_gabriel23
valentina26 maria_luiza31
valentina26 caua11
valentina26 murilo40
v

In [81]:
'alice43' in g.network.get('helena42')

True

# <span style='color:LightSeaGreen'>Funções</span> <a id='functions'></a>

## <span style='color:LightSeaGreen'>Exibir número de seguidores ✔</span> <a id='followers'></a> 

In [19]:
def showFollowers(self, username):
        return f'Seguidores de {self.profiles[username]["Name"]} : {self.profiles[username]["Followers"]}'


## <span style='color:LightSeaGreen'>Exibir pessoas que o usuário segue ✔</span> <a id='following'></a>

In [20]:
def showFollowing(self, username):
        return f'Pessoas que {self.profiles[username]["Name"]} segue: {self.profiles[username]["Following"]}'

## <span style='color:LightSeaGreen'>Ordernar lista stories ✔</span> <a id='stories'></a>

In [21]:
def storiesOrder(self, username):
    firstSort = {key: value for key, value in sorted(self.network[username].items())}
    secondSort = [key for key, _ in sorted(firstSort.items(), reverse=True, key=lambda item: item[1])]
    return f'Ordem dos Stories de {self.profiles[username]["Name"]}:\n {secondSort}'

## <span style='color:LightSeaGreen'>Encontrar top k influencers ✔</span> <a id='topk'></a>

In [22]:
def findInfluencers(self, k):
    # add first node to queue
    queue = [next(iter(self.profiles))]
    # create visited nodes list
    visited = []
    # create influencers dicitonary
    influencers = {}
    # while the list is not empty
    while len(queue) > 0:
        # extract the first element of the queue
        currentNode = queue.pop(0)
        # add this element into the visited list
        visited.append(currentNode)
        # 
        currentFollowerNumber = self.profiles[currentNode]['Followers']
        # verify if list is empty
        if not influencers:
            influencers[currentNode] = currentFollowerNumber
        # get the last element from influencer dict
        leastInfluencer = min(influencers.items(), key=lambda item: item[1])[1]
        # verify if list is full and the new element is bigger
        if len(influencers) == k and currentFollowerNumber > leastInfluencer:
            influencers.popitem()
        # add item and sort values
        if len(influencers) < k :
            influencers[currentNode] = currentFollowerNumber
            influencers = {key: value for key, value in sorted(influencers.items(), reverse=True, key=lambda item: item[1])}

        # visit the adjacent elements of this node
        for adjacent in self.network[currentNode].keys():
            # node not in visited list and in queue
            if adjacent not in queue + visited:
                # add this adjacent node to queue
                queue.append(adjacent)
    return f'Top {k} Influencers: {influencers}'

## <span style='color:LightSeaGreen'>Encontrar o caminho ✔</span> <a id='path'></a>

In [23]:
def showPath(self, origin, destiny):
    # starting node
    queue = [origin]
    # create visited list
    visited = []
    # create predecessor list from origin
    predecessor = {origin: None}
    
    # walk in graph
    while len(queue) > 0:
        # set current node
        currentNode = queue.pop(0)
        # add to visited list
        visited.append(currentNode)
        # visit all adjacents
        for adjacent in self.network[currentNode].keys():
            
            # go trought unvisited nodes
            if adjacent not in queue + visited:
                predecessor[adjacent] = currentNode
                queue.append(adjacent)

            # reach final destination
            if adjacent == destiny:
                
                # create path from predecessor list
                path = [destiny]
                while currentNode is not None:
                    path.append(currentNode)
                    currentNode = predecessor[currentNode]
                path.reverse()
                # format path to print
                result = ['->'] * (len(path) * 2 - 1)
                result[0::2] = path
                fullPath = ' '.join([str(elem) for elem in result])

                return f'Path from {self.profiles[origin]["Name"]} to {self.profiles[destiny]["Name"]}: {fullPath}'

    return False

# <span style='color:LightSeaGreen'>Testes</span> <a id='methods'></a>

quantidade_seguidores('helena42') --> Seguidores da Helena: 18

In [41]:
g.showFollowers('helena42')

'Seguidores de Helena : 18'

quantidade_seguindo('helena42') --> Pessoas que a Helena segue: 16

In [42]:
g.showFollowing('helena42')

'Pessoas que Helena segue: 16'

stories('helena42') --> Ordem dos stories da Helena:  
 ['ana_julia22', 'pietro33', 'alice43', 'ana_clara30', 'calebe49', 'caua11', 'davi48', 'gustavo16', 'heloisa37', 'lavinia36','mariana5', 'matheus6', 'melissa42', 'nicolas4', 'rafael38', 'sophia31']

In [43]:
g.storiesOrder('helena42')

"Ordem dos Stories de Helena:\n ['ana_julia22', 'pietro33', 'alice43', 'ana_clara30', 'calebe49', 'caua11', 'davi48', 'gustavo16', 'heloisa37', 'lavinia36', 'mariana5', 'matheus6', 'melissa42', 'nicolas4', 'rafael38', 'sophia31']"

top_influencers(5) --> Top influences: {'maria_alice19': 24, 'henrique12': 22, 'miguel1': 22, 'isis3': 22, 'alice43': 22}

In [59]:
g.findInfluencers(5)

"Top 5 Influencers: {'helena42': 0, 'alice43': 0, 'gustavo16': 0, 'ana_clara30': 0, 'mariana5': 0}"

mostrar o caminho ('helena42', 'isadora45')

In [46]:
g.showPath('helena42', 'isadora45')

'Path from Helena to Isadora: helena42 -> ana_clara30 -> isadora45'